<h1> Método Monte Carlo - Metropolis </h1>

<!-- En el modelo de Ising 2D, su objetivo es generar configuraciones de espines de acuerdo con la distribución de probabilidad del ensamble canónico:

$$
P(\sigma) = \frac{1}{Z} e^{-\beta E(\sigma)}, \quad \beta = \frac{1}{T}.
$$ -->

1. **Inicialización del sistema**  
   Se crea una red cuadrada de espines $ S_{ij} = \pm 1 $ de tamaño $L \times L$, con valores asignados al azar. Esto corresponde a un estado de "temperatura infinita".

2. **Selección aleatoria de un espín**  
   En cada paso del algoritmo, se elige una posición $(i,j)$ al azar dentro de la red.

3. **Cálculo del cambio de energía**  
   Se evalúa la variación de energía asociada a voltear el espín seleccionado.  
   Debido a que el Hamiltoniano solo involucra primeros vecinos, basta con calcular:

   $$
   \Delta E = E_{\text{nuevo}} - E_{\text{viejo}}
   $$

   que en el modelo de Ising con interacción entre primeros vecinos puede escribirse de forma local como:

   $$
   \Delta E = -2 S_{ij} \sum_{\text{vecinos}} S_{kl}.
   $$

   Este cálculo evita recomputar toda la energía del sistema.

4. **Criterio de aceptación de Metropolis**  

   El cambio de espín se acepta según la regla:

   - Si $ \Delta E \le 0 $, el cambio reduce la energía, por lo que se acepta siempre.
   - Si $ \Delta E > 0 $, el cambio se acepta con probabilidad:

     $$
     P = e^{-\Delta E / T}.
     $$

   Esto permite que el sistema explore estados de energía mayor, evitando quedar atrapado en mínimos locales.

5. **Actualización del estado**  
   Si el cambio es aceptado, el espín se voltea. De lo contrario, se mantiene igual.  
   El procedimiento se repite durante muchos pasos (llamados pasos Monte Carlo).





Para evitar efectos de borde, se implementa una red “toroidal”, donde:

- el borde derecho se conecta con el izquierdo,
- el borde superior con el inferior.

Para generar un conjunto de datos se tendra en cuenta:
- Simulated annealing: Se ejecutan cierto número de pasos, dando un barrido de temperatura de un estado de mayor a menor temperatura, hasta llegar a la temperatura objetivo, esto para evitar quedar en mínimos locales.
- Burn-in (termalización): Se ejecutan muchos pasos sin tomar datos, permitiendo que el sistema alcance equilibrio térmico.
- Muestreo: Una vez alcanzado el equilibrio, se guardan configuraciones espaciadas cada varios pasos, para reducir correlaciones y obtener muestras representativas del ensamble canónico.



In [ ]:
import numpy as np
from numba import njit

class Ising_2D:

    def __init__(self):
        self.T_c = 2 / np.log(1 + np.sqrt(2))
        pass

    # Condiciones de frontera periódicas
    def pbc(self, i):
        """Condiciones de frontera periódicas."""
        if i + 1 > self.L - 1:
            return 0
        elif i - 1 < 0:
            return self.L - 1
        else:
            return i

    # Cálculo de la energía local, interacción de primeros vecinos
    def energy(self, spin, i, j):
        """Calcula la energía local de un espín en la red."""
        return -self.J * spin[i, j] * (
            spin[self.pbc(i - 1), j] +
            spin[self.pbc(i + 1), j] +
            spin[i, self.pbc(j - 1)] +
            spin[i, self.pbc(j + 1)]
        ) + spin[i, j] * self.H

    # Microestado inicial aleatorio (T = ∞, H = 0)
    def build_system(self):
        """Construye una red inicial de espines aleatorios ±1."""
        # spin = np.random.randint(0, 2, (self.L, self.L))
        spin = np.ones((self.L, self.L))
        spin[spin == 0] = -1
        self.spin = spin
    
    def total_energy(self):
        """Calcula la energía total del sistema."""
        energy = 0.0
        # Interacciones horizontales
        energy -= self.J * np.sum(self.spin[:, :-1] * self.spin[:, 1:])
        energy -= self.J * np.sum(self.spin[:, -1] * self.spin[:, 0])  # Condición periódica
        # Interacciones verticales
        energy -= self.J * np.sum(self.spin[:-1, :] * self.spin[1:, :])
        energy -= self.J * np.sum(self.spin[-1, :] * self.spin[0, :])  # Condición periódica
        # Campo magnético
        energy -= self.H * np.sum(self.spin)
        return energy

    def metropolis_hasting(self, STEPS, callback=None):
        """
        Ejecuta el algoritmo de Metropolis-Hastings.
        Si se proporciona un callback, se llama en cada paso con (step, spin, energy, is_sample).
        """
        for step in range(STEPS):
            i = np.random.randint(0, self.L)
            j = np.random.randint(0, self.L)

            Delta_E = -2.0 * self.energy(self.spin, i, j)

            if Delta_E <= 0:
                self.spin[i, j] *= -1
            elif np.exp(-Delta_E / self.T) > np.random.rand():
                self.spin[i, j] *= -1
            
            # Llamar callback si existe
            if callback is not None:
                # Calcular energía total del sistema
                total_energy = self.total_energy()
                callback(step, self.spin.copy(), total_energy, False)

    def generate_samples(self, L, T, samples, burn_in, interval, seed, H, J, folder="./data/"):
  
        file = folder +f"ising_L{L}_T{T:.3f}_{int(T >= self.T_c)}.txt"  # Dataset path
        #// header = f"L={L}; T={T}; N={samples}; class={int(T > self.T_c)}; burn_in={burn_in}; interval={interval}; seed={seed}; H={H}; J={J}" # Header
        header = f'{{"L": {L}, "T": {T}, "N": {samples}, "class": {int(T > self.T_c)}, "burn_in": {burn_in}, "interval": {interval}, "seed": {seed}, "H": {H}, "J": {J}}}'

        with open(file, "w") as f:
            f.write(header + "\n")
        
        # Global variables
        self.L = L
        self.J = J
        self.H = H

        self.build_system()  # Generar el microestado inicial
        
        # Simulated annealing 
        for T_annealing in np.arange(T, 5, 0.05)[::-1]:
            self.T = T_annealing
            self.metropolis_hasting(100) 
        
        # Burn_in
        self.T = T
        self.metropolis_hasting(burn_in) 

        # Sampling
        for i in range(samples):
            self.metropolis_hasting(interval) # Realiza el muestreo
            with open(file, "a") as f:               # Escribe el estado actual en el archivo
                f.write(','.join(map(str, self.spin.flatten())))
                f.write('\n')
            
# ising = Ising_2D()
# ising.generate_samples(L=10, T=0.5, samples=2, burn_in=1000, interval=100, seed=42, H=0, J=1, folder='./../data/')

📁 Sistema de archivos de `generate_samples`

El método `generate_samples(...)` genera un archivo `.txt` por cada temperatura \(T\) y tamaño \(L\).  
Cada archivo contiene:

- Un header en la primera línea (metadatos del experimento)
- Una lista de microestados planos (cada línea corresponde a un microestado)

El nombre del archivo se construye así: `ising_L{L}_T{T:.3f}_{label}.txt`

Donde

| Campo | Significado |
|-------|-------------|
| `L` | Tamaño del sistema $L \times L$ |
| `T` | Temperatura con 3 decimales |
| `N` | Número de muestras generadas |
| `label` | Etiqueta de fase (0 = ordenada, 1 = desordenada) |

La primera línea del archivo contiene los paramétros con los que se ha generado la muestra:

`{"L":10, "T":2.3, "N":5000, "class":1, "burn_in":2000, "interval":100, "seed":42, "H":0, "J":1}`

Parámetros de simulación metropolis_hasting: `burn_in`, `interval`

Etiqueta de fase: `class`

Semilla del generador aleatorio: `seed`

El directorio se especifica mediante `folder`


In [ ]:
from tqdm import tqdm
T = np.arange(0.1, 4.1, 0.1)

ising = Ising_2D()


for t in tqdm(T):
    ising.generate_samples(L=10, T=t, samples=2000, burn_in=5000, interval=500, seed=73, H=0, J=1, folder='./data/data_30/')

  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [06:16<00:00,  9.41s/it]
